# About Dataset

Context This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content 5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset. Columns

* asin - ID of the product, like B000FA64PK
* helpful - helpfulness rating of the review - example: 2/3.
* overall - rating of the product.
* reviewText - text of the review (heading).
* reviewTime - time of the review (raw).
* reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
* reviewerName - name of the reviewer.
* summary - summary of the review (description).
* unixReviewTime - unix timestamp.

Acknowledgements This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration

* Sentiment analysis on reviews.
* Understanding how people rate usefulness of a review/ What * factors influence helpfulness of a review.
* Fake reviews/ outliers.
* Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
* Any other interesting analysis
##  Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [1]:
# Load the dataset
import pandas as pd
data=pd.read_csv("all_kindle_review.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [2]:
data= data[['reviewText','rating']]
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [3]:
data.shape

(12000, 2)

In [4]:
## Missing Values
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [5]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [6]:
data['rating'].value_counts()

5    3000
4    3000
3    2000
2    2000
1    2000
Name: rating, dtype: int64

In [7]:
## Preprocessing and Cleaning

In [8]:
#Positive review is 1 and Negative review is 0
data['rating']= data['rating'].apply(lambda x:0 if x<3 else 1)

In [9]:
data['rating'].value_counts()

1    8000
0    4000
Name: rating, dtype: int64

In [10]:
## 1.Lower all the cases
data['reviewText'] = data['reviewText'].str.lower()

In [11]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [12]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from bs4 import BeautifulSoup

In [14]:
##2.Removing speacial characters

data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

C:\Users\admin\AppData\Local\Temp\ipykernel_8100\1626603280.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())


In [15]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [16]:
## Lemmatizer
from nltk.stem import WordNetLemmatizer

In [17]:
lemmatizer=WordNetLemmatizer()

In [18]:
def lemmatizer_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])


In [20]:
data['reviewText']=data['reviewText'].apply(lambda x:lemmatizer_words(x))

In [21]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [24]:
## Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(data['reviewText'],data['rating'],test_size=0.20)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [27]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)

In [29]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [30]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [31]:
y_pred_tfidf=nb_model_bow.predict(X_test_tfidf)

In [32]:
confusion_matrix(y_test,y_pred_bow)

array([[499, 299],
       [739, 863]], dtype=int64)

In [33]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.5675


In [34]:
confusion_matrix(y_test,y_pred_tfidf)

array([[492, 306],
       [736, 866]], dtype=int64)

In [35]:
print("TFIDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TFIDF accuracy:  0.5658333333333333


## Using Word2Vec

In [45]:
print(data.columns)

Index(['reviewText', 'rating'], dtype='object')


In [46]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

corpus = []
for text in data['reviewText']:  # Use 'reviewText' as the column name
    # Preprocess the text data
    text = re.sub('[^a-zA-Z]', ' ', text)  # Remove non-alphabet characters
    text = text.lower()
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    corpus.append(lemmatized_tokens)


In [47]:
from gensim.models import Word2Vec

# Initialize and train Word2Vec
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, workers=4)

In [48]:
import numpy as np

def get_sentence_vector(sentence, model):
    # Get vectors for words in the sentence that exist in the model's vocabulary
    words = [model.wv[word] for word in sentence if word in model.wv]
    if len(words) > 0:
        return np.mean(words, axis=0)  # Average word vectors to get sentence vector
    else:
        return np.zeros(model.vector_size)  # Return a zero vector if no words found

# Generate sentence vectors for the entire corpus
X_word2vec = np.array([get_sentence_vector(sentence, word2vec_model) for sentence in corpus])

In [50]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_word2vec,data['rating'], test_size=0.2, random_state=42)

In [51]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [52]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Predict on the test set
y_pred = classifier.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7166666666666667
Confusion Matrix:
 [[ 591  212]
 [ 468 1129]]
Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.74      0.63       803
           1       0.84      0.71      0.77      1597

    accuracy                           0.72      2400
   macro avg       0.70      0.72      0.70      2400
weighted avg       0.75      0.72      0.72      2400

